In [1]:
import numpy as np

import trimesh

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


## Data

Load triangle mesh

In [2]:
# Surface Mesh
mesh = trimesh.load_mesh(file_obj="11d.obj", file_type="obj")

In [3]:
# Visualize Mesh
scene = trimesh.Scene()
scene.add_geometry(mesh)
png = scene.save_image()

with open("figures/11d.png", "wb") as file:
	file.write(png)

# display
mesh.show()

## Surface Curvature

Measure surface curvature

In [4]:
# Calculate Curvature
gauss = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)
mean = trimesh.curvature.discrete_mean_curvature_measure(mesh, mesh.vertices, 1)


In [5]:
# Determine Colors
k0 = min(np.abs(gauss))
k1 = max(np.abs(gauss))

def color(name:str, l:float=1.0) -> list[float]:

    ''' Get RGB of given color with adjusted lightness 
    
    Parameters
    ----------
    name : str
        base color name
    l : float (default = None)
        lightness/value (for HSV) \\
        if None, use lightness of {name}
    
    Returns
    -------
    _ : list[float]
        red, green, and blue values [0,1]
    '''

    rgb = mcolors.to_rgb(name)
    hsv = mcolors.rgb_to_hsv(rgb)
    
    if l:
        hsv[2] = l
    
    return mcolors.hsv_to_rgb(hsv)

colors = [0] * len(mean)

# for each vertex:
for i,(k,h) in enumerate(zip(gauss, mean)):

    # define lightness
    l = 0.5 + (0.5 * (abs(k) - k0) / (k1 - k0))

    # convex
    if (k > 0) and (h < 0):
        colors[i] = color("green", l)

    # concave
    elif (k > 0) and (h > 0):
        colors[i] = color("red", l)
    
    # hyperbolic
    elif (k < 0):
        colors[i] = color("lightgray", l)
    
    # cylindrical
    else:
        colors[i] = color("black", l)

In [6]:
# Visualize Curvature Regions
mesh.visual.vertex_colors = colors

# save
scene = trimesh.Scene()
scene.add_geometry(mesh)
png = scene.save_image()

with open("figures/curvature_regions.png", "wb") as file:
	file.write(png)

# display
mesh.show()